#OpenAI GTP-2
Copyright 2020, Denis Rothman MIT License. Denis Rothman created the Colab notebook using the OpenAI repository, adding title steps for educational purposes only.

It is important to note that we are running a low-level GPT-2 model 
and not a one-line call to obtain a result. We are also
avoiding pre-packaged versions. We are getting our hands dirty to
understand the architecture of a GPT-2 from scratch. You might get
some deprecation messages. However, the effort is worthwhile.

***Code Reference***
[Reference: OpenAI Repository](https://github.com/openai/gpt-2)

***Model Reference***
[Alec Radford, Jeffrey Wu, Rewon Child, David Luan, Dario Amodei, Ilya Sutskever,2019,'Language Models are Unsupervised Multitask Learners'](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf)


Step 1: Pre-requisite: activate GPU in the notebook settings runTime menu



In [1]:
#@title Step 2: Cloning the OpenAI GPT-2 Repository 
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 233, done.
remote: Total 233 (delta 0), reused 0 (delta 0), pack-reused 233
Receiving objects: 100% (233/233), 4.38 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [2]:
#@title Step 3: Installing the requirements
import os                     # when the VM restarts import os necessary
os.chdir("/content/gpt-2")    
!pip3 install -r requirements.txt

     |████████████████████████████████| 92kB 6.2MB/s 
     |████████████████████████████████| 604kB 13.4MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=9b8cd96fb78bc8bc741c1ba90bcfa8f52381e1c25e5ad8bb58ed6a30967e51f6
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
  Created wheel for regex: filename=regex-2017.4.5-cp37-cp37m-linux_x86_64.whl size=534388 sha256=0179b2ca2a05b094eda38db96ad802be0a19b4a36483e53e65deb0690c134ad7
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.31.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but

In [1]:
#@title Step 4 Checking the Version of TensorFlow 
#Colab has tf 1.x and tf 2.x installed
#Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [2]:
#@title Step 5: Downloading the 345M parameter GPT-2 Model
# run code and send argument
import os # after runtime is restarted
os.chdir("/content/gpt-2")
!python3 download_model.py '345M' 

Fetching checkpoint: 1.00kit [00:00, 755kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 1.46Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 723kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [07:22, 3.20Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 7.53Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 1.46Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 890kit/s]                                                        


In [3]:
#@title Step 6: Printing UTF encoded text to the console
!export PYTHONIOENCODING=UTF-8

In [4]:
#@title Step 7: Project Source Code
import os # import after runtime is restarted
os.chdir("/content/gpt-2/src")

In [5]:
#@title Step 7a: Interactive Conditional Samples (src)
#Project Source Code for Interactive Conditional Samples:
# /content/gpt-2/src/interactive_conditional_samples.py file 
import json
import os
import numpy as np
import tensorflow as tf

In [6]:
#@title Step 7b: Importing model sample encoder
import model, sample, encoder
#if following message:
#ModuleNotFoundError: No module named 'tensorflow.contrib'
#then go back and run Step 2 Checking TensorFlow version 

In [7]:
#@title Step 8: Defining the model
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [8]:
#@title Step 9: Interacting with GPT-2 
interact_model('345M',None,1,1,300,1,0,'/content/gpt-2/models')





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> Human reason, in one sphere of its cognition, is called upon to consider questions, which it cannot decline, as they are presented by its own nature, but which it cannot answer, as they transcend every faculty of the mind.
======================================== SAMPLE 1 ========================================
 But to set religion back under that cursed vale of tears is naturally to hinder its penetrating into itself. For either the truth of the union of man and man is able to be touched under the feet of the naked self, or else that union is brought to the p161 light by blindness, havying no use; both of these must be admitted. In refusing to confess that the flesh gave history, 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/ipykernel/kernelbase.py", line 729, in _input_request
    ident, reply = self.session.recv(self.stdin_socket, 0)
  File "/usr/local/lib/python3.7/dist-packages/jupyter_client/session.py", line 803, in recv
    msg_list = socket.recv_multipart(mode, copy=copy)
  File "/usr/local/lib/python3.7/dist-packages/zmq/sugar/socket.py", line 583, in recv_multipart
    parts = [self.recv(flags, copy=copy, track=track)]
  File "zmq/backend/cython/socket.pyx", line 781, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 817, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 186, in zmq.backend.cython.socket._recv_copy
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/li

TypeError: ignored